In [435]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

# Data Cleaning

In [436]:
# raw_data是从原表读到的原始数据
raw_data = pd.read_excel(os.path.join(os.getcwd(),'../data/rawdata/car_all_beijing.xls'))

In [437]:
raw_data = raw_data.rename(columns={
    '标题':'title',
    '价格':'price',
    '新车价格':'price_new',
    '首次上牌':'date_regi',
    '表显里程':'mileage', 
    '官方续航':'official_endurance',
    '排放标准':'standard',
    '变速箱':'gearbox',
    '过户次数':'num_trans',
    '车牌地':'license_location',
    '车身颜色':'color',
    '电动机总功率':'motor_power',
    '电池容量':'battery_capacity',
    '电池类型':'battery_type',
    '能源类型':'energy_type',
    '排量':'displacement',
    '钥匙数':'keys'
    })

In [438]:
print(raw_data.shape) # (39216, 17)
# 根据title去掉空行
not_null_raw_data = raw_data[~(raw_data['title'].isnull())]
print(not_null_raw_data.shape) # (39159, 17)
print('空行数量：',39216-39159)
print('查看每个列空值数量：')
print(not_null_raw_data.isnull().sum())

(19361, 17)
(19300, 17)
空行数量： 57
查看每个列空值数量：
title                     0
price                     0
price_new                 0
date_regi                 0
mileage                   0
official_endurance    14296
standard                  0
gearbox                   0
num_trans                 0
license_location          0
color                     0
motor_power           14296
battery_capacity      14296
battery_type          14296
energy_type               0
displacement           5004
keys                      0
dtype: int64


In [439]:
# 全部数据清洗
clean_data = not_null_raw_data.copy()
# title处理
clean_data['brand'] = clean_data['title'].str.split(' ',expand=True)[0]

In [440]:
#去掉-
clean_data['brand']=clean_data['brand'].map(lambda x: re.sub(r'[-·]','',x))

In [441]:
#去掉字母数字
def find_brand(x):
    """brand"""
    if re.search(r'[\u4e00-\u9fff]+', x):
        res = re.sub(r'( )*[0-9A-Za-z]+$','',x)
    else:
        res = x
    return res
clean_data['brand'] = clean_data['brand'].apply(find_brand)
clean_data['brand']

0          荣威
1        理想汽车
2          丰田
3          宝骏
4        零跑汽车
         ... 
19356    北京汽车
19357      哈弗
19358    雷克萨斯
19359    凯迪拉克
19360    小鹏汽车
Name: brand, Length: 19300, dtype: object

In [442]:
clean_data['brand'].isnull().sum()

0

In [443]:
clean_data['brand'].unique()

array(['荣威', '理想汽车', '丰田', '宝骏', '零跑汽车', '威马汽车', '别克', '现代', '日产', '大众',
       '起亚', '奇瑞', '特斯拉', '本田', '北京', '沃尔沃', 'WEY', '吉利汽车', '奔驰C级',
       '奔驰GLC级', '广汽传祺', '长安', 'MINI', '马自达', '上汽大通', '蔚来', '雷克萨斯', '比亚迪',
       '宝马', '捷达', '北汽新能源', '欧拉黑猫', '哪吒汽车', '领克', '新宝骏', 'Jeep', '五菱汽车',
       '凯迪拉克', '宝马3系', '几何汽车', '哈弗大狗', '奥迪', '广汽埃安', '长安欧尚', '理念', '路虎',
       '斯柯达', '小鹏汽车', '哈弗', '思皓', '红旗', '奔驰E级', '宝马5系', '保时捷', '捷豹',
       '东风风行', '名爵', '福特', '东风风神', '奔驰A级', '欧拉好猫', '林肯', '长城', '雪佛兰',
       '合众汽车', '哈弗神兽', '奔腾', '林肯大陆', '宝马1系', '玛莎拉蒂', '云度π', '本田e:',
       '英菲尼迪G系列', '腾势', '江淮', '星途凌云', '东风风光', '启辰', '三菱', '大众ID.',
       '奔驰CLA级', '哈弗初恋', '国机智骏', '岚图汽车', '宝马7系', '上汽', '思铭', '众泰',
       '本田CRV新能源', '启辰大', '奇瑞新能源', '雷克萨斯UX新能源', '奇瑞蚂蚁', '北京汽车', '坦克',
       '长安跨越', '长安新能源', '广汽新能源', '高合汽车', '奔驰GLA级', '奔驰V级', '宝马2系', '飞凡汽车',
       '极氪', '欧拉白猫', '知豆', '奔驰GLE级', '观致', '凌宝汽车', '启辰星', '宝马5系新能源',
       '斯巴鲁', 'AITO', '天际汽车', '众泰T300新能源', '宝马2系多功能旅行车', '雪铁龙', '宝马6系',
       '奔驰'

In [444]:
#去掉-
clean_data['brand']=clean_data['brand'].map(lambda x: re.sub(r'[-·]','',x))

# 手动去掉北京品牌中的多余字
clean_data['brand']=clean_data['brand'].map(
     lambda x: re.sub(r'[级系]+','', x)).map(
     lambda x: re.sub(r'(风光)|(风行)|(风神)|(小康)|(EV)|(新能源)|(黑猫)|(传祺)|(欧尚)|(罗密欧)|(2多功能旅行车)|(2旅行车)|(ID.)|(e:)|(G列)|(大狗)|(初恋)|(EZS纯电动)|(跨越)|(白猫)|(蚂蚁)|(好猫)|(精灵#)|(轻型车)|(原力版)|(神兽)|(赤兔)|(CRV)|(UX)|(T300)|(Panamera)|(S90)|(UNIK)|(DX3)|(HS)|(XC60)|(A6L)|(X40)|(C30)','', x)).map(
     lambda x: re.sub(r'(启辰大|启辰星)','启辰', x)).map(
     lambda x: re.sub(r'(BEIJING汽车)','北京汽车', x)).map(
     lambda x: re.sub(r'(奔驰E)|(奔驰B)|(奔驰S)|(奔驰GLA)|(奔驰A)|(奔驰V)|(奔驰GLS)|(奔驰SLK)|(奔驰CLA)|(奔驰GLE)|(奔驰GLC)|(奔驰GLK)|(奔驰CLS)|(奔驰LC)|(奔驰R)|(奔驰C)','奔驰', x)).map(
      lambda x: re.sub(r'(领克01)|(领克06)|(领克05)|(领克09)','领克', x)).map(
      lambda x: re.sub(r'(宝马5)|(宝马2)|(宝马1)|(宝马X1新能源)|(宝马4)|(宝马X5)|(宝马7)|(宝马6)|(宝马3)|(宝马X1)','宝马', x))


In [445]:
#去掉-
clean_data['brand']=clean_data['brand'].map(lambda x: re.sub(r'[-·]','',x))

# 手动去掉深圳品牌中的多余字
clean_data['brand']=clean_data['brand'].map(
     lambda x: re.sub(r'[级系]+','', x)).map(
     lambda x: re.sub(r'(风光)|(风行)|(风神)|(小康)|(EV)|(新能源)|(黑猫)|(传祺)|(欧尚)|(罗密欧)|(2多功能旅行车)|(2旅行车)|(ID.)|(e:)|(G列)|(大狗)|(初恋)|(EZS纯电动)|(跨越)|(白猫)|(蚂蚁)|(好猫)|(精灵#)|(轻型车)|(原力版)|(神兽)|(赤兔)|(埃安)|(埃安)|(酷狗)|(风度)|(S5青春版)|(昌河)|(制造)|(ATSL2017款)|(绅宝)|(Macan2018款)|(3星骋)|(ATSL2017款)|(RX经典)|(2018款)|(A5翼舞)|(电马)|(集团)','', x)).map(
     lambda x: re.sub(r'(启辰大|启辰星)','启辰', x)).map(
     lambda x: re.sub(r'(BEIJING汽车)','北京汽车', x)).map(
     lambda x: re.sub(r'(名爵6)','名爵', x))


In [446]:

# 手动去掉shanghai品牌中的多余字
clean_data['brand']=clean_data['brand'].map(
     lambda x: re.sub(r'[级系]+','', x)).map(
     lambda x: re.sub(r'(风光)|(风行)|(风神)|(小康)|(新能源)|(黑猫)|(传祺)|(欧尚)|(罗密欧)|(2多功能旅行车)|(2旅行车)|(ID.)|(e:)|(G列)|(大狗)|(初恋)|(EZS纯电动)|(跨越)|(白猫)|(蚂蚁)|(好猫)|(精灵#)|(轻型车)|(原力版)|(神兽)|(赤兔)|(埃安)|(埃安)|(酷狗)|(风度)|(S5青春版)|(昌河)|(制造)|(福克斯2018款)|(CT6)|(S60)|(CRV)|(UNIK)|(A6L)|(UX)|(3星骋)|(T300)|(up!)|(S0)|(CT)','', x)).map(
     lambda x: re.sub(r'(启辰大|启辰星)','启辰', x)).map(
     lambda x: re.sub(r'(BEIJING汽车)','北京汽车', x)).map(
     lambda x: re.sub(r'(宝马5)|(宝马1)|(宝马X1新能源)|(宝马4)|(宝马X5)|(宝马7)|(宝马6)|(宝马3)|(宝马X1)','宝马', x)).map(
     lambda x: re.sub(r'(奔驰E)|(奔驰B)|(奔驰S)|(奔驰GLA)|(奔驰A)|(奔驰V)|(奔驰GLS)|(奔驰SLK)|(奔驰CLA)|(奔驰GLE)|(奔驰GLC)|(奔驰GLK)|(奔驰CLS)|(奔驰LC)|(奔驰R)|(奔驰C)|(奔驰M)|(奔驰LK)','奔驰', x)).map(
     lambda x: re.sub(r'(领克01)|(领克06)|(领克05)|(领克03)|(领克0)','领克', x)).map(
     lambda x: re.sub(r'(沃尔沃L)','沃尔沃', x))



In [448]:
clean_data['brand'].unique()

array(['荣威', '理想汽车', '丰田', '宝骏', '零跑汽车', '威马汽车', '别克', '现代', '日产', '大众',
       '起亚', '奇瑞', '特斯拉', '本田', '北京', '沃尔沃', 'WEY', '吉利汽车', '奔驰', '广汽',
       '长安', 'MINI', '马自达', '上汽大通', '蔚来', '雷克萨斯', '比亚迪', '宝马', '捷达', '北汽',
       '欧拉', '哪吒汽车', '领克', '新宝骏', 'Jeep', '五菱汽车', '凯迪拉克', '几何汽车', '哈弗',
       '奥迪', '理念', '路虎', '斯柯达', '小鹏汽车', '思皓', '红旗', '保时捷', '捷豹', '东风',
       '名爵', '福特', '林肯', '长城', '雪佛兰', '合众汽车', '奔腾', '林肯大陆', '玛莎拉蒂', '云度π',
       '英菲尼迪', '腾势', '江淮', '星途凌云', '启辰', '三菱', '国机智骏', '岚图汽车', '上汽', '思铭',
       '众泰', '北京汽车', '坦克', '高合汽车', '飞凡汽车', '极氪', '知豆', '观致', '凌宝汽车',
       '斯巴鲁', 'AITO', '天际汽车', '雪铁龙', '标致', 'ARCFOX极狐', '凯翼', '江铃', '星途追风',
       'SERES赛力斯', '海马', '创维汽车', '全球鹰', 'R汽车', '捷途', '百智', '雷丁芒果', '合创',
       '宝沃', '五十铃', '福田', '金杯', '讴歌', 'LITE', 'Polestar', '新特汽车', '雷诺',
       'SWM斯威汽车', '东南', '枫叶汽车', '金杯海狮王', '猎豹汽车', '电咖', 'smart', 'HYCAN合创',
       '野马汽车', '克莱斯勒', '天美汽车', '朋克汽车', '大运', '华晨新日', '电动屋', '星途揽月',
       '长安凯程', 'DS', '吉利', '海马爱尚', '汉腾汽车', '中华', '潍柴汽车'

In [449]:
clean_data['brand'].isnull().sum()

0

In [450]:
# 去掉单位
clean_data['mileage'] = clean_data['mileage'].str.replace('万公里','').apply(lambda x :float(x.replace('公里',''))/10000 if '公里' in str(x) else x).astype(np.float64)
clean_data['official_endurance'] = clean_data['official_endurance'].str.replace('km','').replace('-',np.nan).astype(np.float64)
clean_data['num_trans'] = clean_data['num_trans'].str.replace('次','').replace('-',np.nan).astype(np.float64)
clean_data['battery_capacity'] = clean_data['battery_capacity'].str.replace('kWh','').replace('-',np.nan).astype(np.float64)
clean_data['motor_power'] = clean_data['motor_power'].str.replace('kw','').replace('-',np.nan).astype(np.float64)
clean_data['keys'] = clean_data['keys'].str.replace('把','').astype(np.float64)

# location 一线城市（北上广深）为1，其他城市为0
clean_data['license_location'] = clean_data['license_location'].str.split('[(（]',expand=True)[0]
clean_data['license_location'] = clean_data['license_location'].apply(lambda x : 1 if x in ['上海','北京','广州','深圳'] else 0).astype(np.float64)

# color 黑色、白色、深灰色、银灰色为0，其他颜色为1
clean_data['color'] = clean_data['color'].apply(lambda x : 0 if x in ['白色','黑色','深灰色','银灰色'] else 1).astype(np.float64)

# displace：T*1.4变成L，方便分析，排放越大性能越好
clean_data['displacement'] = clean_data['displacement'].astype(str).apply(lambda x : float(x.replace('T',''))*1.4 if 'T' in x else float(x.replace('L',''))).astype(np.float64)


In [451]:
energy_type = list(clean_data['energy_type'].dropna().unique())
energy_type

['汽油',
 '增程式',
 '纯电动',
 '汽油+48V轻混系统',
 '插电式混合动力',
 '油电混合',
 '柴油',
 '汽油电驱',
 '汽油+24V轻混系统']

In [452]:
# 能源类型:汽油0，电动1，混合2
energy_type = list(clean_data['energy_type'].dropna().unique())
# 北京
type_num = [0,2,1,2,2,2,2,0,2,2]

#深圳
# type_num = [2,0,1,2,2,2,2,0,2,2]

#上海
# type_num = [2,0,1,2,2,2,2,0,2]
energy_type_num=dict(zip(energy_type,type_num))
clean_data['energy_type'] = clean_data['energy_type'].map(energy_type_num)

# 变速箱：手动0，自动1
clean_data['gearbox'] = clean_data['gearbox'].apply(lambda x : 1 if str(x)=='自动' else 0)

In [453]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19300 entries, 0 to 19360
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               19300 non-null  object 
 1   price               19300 non-null  float64
 2   price_new           19300 non-null  float64
 3   date_regi           19300 non-null  object 
 4   mileage             19300 non-null  float64
 5   official_endurance  4998 non-null   float64
 6   standard            19300 non-null  object 
 7   gearbox             19300 non-null  int64  
 8   num_trans           19300 non-null  float64
 9   license_location    19300 non-null  float64
 10  color               19300 non-null  float64
 11  motor_power         5003 non-null   float64
 12  battery_capacity    4849 non-null   float64
 13  battery_type        5004 non-null   object 
 14  energy_type         19300 non-null  int64  
 15  displacement        14296 non-null  float64
 16  keys

In [454]:
# 挑出油车
petrol = clean_data[clean_data['energy_type']==0]
# 删除电车特征
petrol = petrol.drop(['official_endurance','motor_power','battery_capacity','battery_type'],axis=1)

In [455]:
#挑出电车
electric = clean_data[clean_data['energy_type']==1]
# 均值motor_power、battery_capacity、official_endurance # 众数battery_type #删除displacement
electric['motor_power'].fillna(electric['motor_power'].mean(), inplace=True)
electric['battery_capacity'].fillna(electric['battery_capacity'].mean(), inplace=True)
electric['official_endurance'].fillna(electric['official_endurance'].mean(), inplace=True)
electric['battery_type'].fillna('三元锂电池', inplace=True)
electric = electric.drop(['displacement'],axis=1)

/var/folders/hv/0r6f95ls1ss09znhzxqsj0nr0000gn/T/ipykernel_73808/2910221480.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electric['motor_power'].fillna(electric['motor_power'].mean(), inplace=True)
/var/folders/hv/0r6f95ls1ss09znhzxqsj0nr0000gn/T/ipykernel_73808/2910221480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electric['battery_capacity'].fillna(electric['battery_capacity'].mean(), inplace=True)
/var/folders/hv/0r6f95ls1ss09znhzxqsj0nr0000gn/T/ipykernel_73808/2910221480.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [456]:
#挑出混合车
mixed = clean_data[clean_data['energy_type']==2]

In [457]:
# 写文件
def write_xlsx(df,path, sheetName):
    writer = pd.ExcelWriter(path, engine='openpyxl',mode='a', if_sheet_exists="overlay")
    df.to_excel(writer, sheet_name=sheetName)
    writer.close()

In [458]:
# 写入同一个文件三张表
path = '../data/cleaneddata/cleaned_car_all_beijing.xlsx'
pd.DataFrame().to_excel(path,sheet_name='total') # 先创建文件
write_xlsx(petrol,path,'petrol')
write_xlsx(electric, path,'electric')
write_xlsx(mixed,path,'mixed')
write_xlsx(clean_data, path,'total')